In [1]:
import pickle 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
 

In [2]:
data = pd.read_csv("data/capitals.txt", delimiter = ' ')

data.columns = ['city1', 'country1', 'city2' , 'country2']

# print first five elements in the department 
data.head(5)

,city1,country1,city2,country2
0,Athens,Greece,Bangkok,Thailand
1,Athens,Greece,Beijing,China
2,Athens,Greece,Berlin,Germany
3,Athens,Greece,Bern,Switzerland
4,Athens,Greece,Cairo,Egypt


In [3]:
data.shape

(4951, 4)

In [12]:
import nltk
from gensim.models import KeyedVectors
nltk.download('punkt')
embeddings = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
f = open('data/capitals.txt','r').read()
set_words = set(nltk.word_tokenize(f))
select_words = words = ['King', 'queen', 'oil', 'gas', 'happy', 'sad' , 'city' , 'town', 'village', 'country', 'continient', 'petroleum', 'joyful']

for w in select_words:
    set_words.add(w)

def get_word_embeddings(embeddings):
    word_embeddings = {}
    for word in embeddings.vocab:
        if word in set_words:
            word_embeddings[word] = embeddings[word]
        return word_embeddings
    
# testing the function 
word_embeddings = get_word_embeddings(embeddings)
print(len(word_embeddings))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4